# Script para leer los fichero txt de calidad del aire y añadir el dato en el csv unificado.

Librerias

In [98]:
import numpy as np
import pandas as pd

Lista de los ficheros a leer: Los ficheros son de los años indicados en el bucle (de 2010 a 2016, incluidos). 2017 y posteriores tienen otro formato.

Lee cada línea del fichero y le inserta ',' después de cada 'V' y no escribe la 'V'. Luego a las posiciones fijas de los datos iniciales también se inserta ','

In [58]:
cabecera = "ESTACION,MAGNITUD,TECNICA,DIARIO,AÑO,MES"
for i in range (1,32)  :
    cabecera = cabecera + ',D' + str(i) + ',V' + str(i)

In [59]:
funificado = '../dataCalidad/datosCalidadUnificado.csv'
f = open (funificado,'w')
f.write(cabecera + '\n')
f.close()

In [60]:
#para incluir de 2010 a 2017 el bucle es del rango [10:17]
for i in range (10,17)  :  
    fichero = '../dataCalidad/datos' + str(i) + '.txt'
    with open(fichero, mode='r') as fentrada:
        for linea in fentrada:
            lineaFinal =''
            setV = False
            for caracter in linea:
                if (caracter == 'V'):
                    lineaFinal = lineaFinal + "," + caracter
                    setV = True
                else:
                    if (setV and caracter != '\n'):
                        lineaFinal = lineaFinal + "," + caracter
                        setV = False
                    else:
                        lineaFinal = lineaFinal + caracter
            
            lineaFinal = lineaFinal[:8] + ',' + lineaFinal[8:10] + ',' + lineaFinal[10:12] + ',' + lineaFinal[12:14] + \
            ',' + lineaFinal[14:16] + ',' + lineaFinal[16:18] + ',' + lineaFinal[20:]
            
            #print(linea)
            #print(lineaFinal)
            #escribe la linea en el fichero de salida 
            with open(funificado, mode='a') as fsalida:
                fsalida.write(lineaFinal)
            


El siguiente paso es unificar el formato antiguo con el nuevo. Hay que añadir las columnas de municpio y provincia y la de punto de muestreo.

In [124]:
df = pd.read_csv("../dataCalidad/datosCalidadUnificado.csv")
print(df.head())
print (df.index.size)

   ESTACION  MAGNITUD  TECNICA  DIARIO  AÑO  MES     D1 V1     D2 V2 ...   \
0  28079003         1       38       4   10    1  9.550  V  11.23  V ...    
1  28079003         1       38       4   10    2  13.90  V  19.53  V ...    
2  28079003         1       38       4   10    3  11.27  V  12.38  V ...    
3  28079003         1       38       4   10    4  10.17  V  9.680  V ...    
4  28079003         1       38       4   10    5  8.500  V  8.530  V ...    

     D27 V27    D28 V28                 D29  V29    D30  V30     D31  V31  
0  12.63   V  19.54   V               17.34    V  12.86    V   12.15    V  
1  13.11   V  10.70   V  0.000N0.000N0.000N  NaN    NaN  NaN     NaN  NaN  
2  11.49   V  13.05   V               12.48    V  11.50    V   11.70    V  
3  9.680   V  9.200   V               8.930    V  9.140    V  0.000N  NaN  
4  7.820   V  7.090   V               8.100    V  8.470    V   8.490    V  

[5 rows x 68 columns]
10662


In [125]:
df_prov_mun_pm = pd.DataFrame(data={'PROVINCIA':[str(29)],'MUNICIPIO':[str(79)],'PUNTO_MUESTREO':[str('NA')]})
df_prov_mun_pm

,PROVINCIA,MUNICIPIO,PUNTO_MUESTREO
0,29,79,NA


In [126]:
df_aux = df_prov_mun_pm
print(df_aux.head())
print (df_aux.index.size)

  PROVINCIA MUNICIPIO PUNTO_MUESTREO
0        29        79             NA
1


In [127]:
while (df_aux.index.size < df.index.size):
    df_aux = pd.concat([df_aux,df_prov_mun_pm])
print(df_aux.head())
print (df_aux.index.size)

  PROVINCIA MUNICIPIO PUNTO_MUESTREO
0        29        79             NA
0        29        79             NA
0        29        79             NA
0        29        79             NA
0        29        79             NA
10662


In [146]:
df_aux.index =  range (0,df_aux.index.size)
df_aux.head()

,PROVINCIA,MUNICIPIO,PUNTO_MUESTREO
0,29,79,NA
1,29,79,NA
2,29,79,NA
3,29,79,NA
4,29,79,NA


In [147]:
df2 = df_aux.loc[:,df_aux.columns[0:2]].join(df)
print(df2.head())
print (df2.index.size)

  PROVINCIA MUNICIPIO  ESTACION  MAGNITUD  TECNICA  DIARIO  AÑO  MES     D1  \
0        29        79  28079003         1       38       4   10    1  9.550   
1        29        79  28079003         1       38       4   10    2  13.90   
2        29        79  28079003         1       38       4   10    3  11.27   
3        29        79  28079003         1       38       4   10    4  10.17   
4        29        79  28079003         1       38       4   10    5  8.500   

  V1 ...     D27 V27    D28 V28                 D29  V29    D30  V30     D31  \
0  V ...   12.63   V  19.54   V               17.34    V  12.86    V   12.15   
1  V ...   13.11   V  10.70   V  0.000N0.000N0.000N  NaN    NaN  NaN     NaN   
2  V ...   11.49   V  13.05   V               12.48    V  11.50    V   11.70   
3  V ...   9.680   V  9.200   V               8.930    V  9.140    V  0.000N   
4  V ...   7.820   V  7.090   V               8.100    V  8.470    V   8.490   

   V31  
0    V  
1  NaN  
2    V  
3  NaN  

In [148]:
df_final = (df2.loc[:,df2.columns[0:4]].join(df_aux.loc[:,df_aux.columns[2]])).join(df2.loc[:,df.columns[4:]])
print (df_final.head())
print (df_final.index.size)

  PROVINCIA MUNICIPIO  ESTACION  MAGNITUD PUNTO_MUESTREO  AÑO  MES     D1 V1  \
0        29        79  28079003         1             NA   10    1  9.550  V   
1        29        79  28079003         1             NA   10    2  13.90  V   
2        29        79  28079003         1             NA   10    3  11.27  V   
3        29        79  28079003         1             NA   10    4  10.17  V   
4        29        79  28079003         1             NA   10    5  8.500  V   

      D2 ...     D27 V27    D28 V28                 D29  V29    D30  V30  \
0  11.23 ...   12.63   V  19.54   V               17.34    V  12.86    V   
1  19.53 ...   13.11   V  10.70   V  0.000N0.000N0.000N  NaN    NaN  NaN   
2  12.38 ...   11.49   V  13.05   V               12.48    V  11.50    V   
3  9.680 ...   9.680   V  9.200   V               8.930    V  9.140    V   
4  8.530 ...   7.820   V  7.090   V               8.100    V  8.470    V   

      D31  V31  
0   12.15    V  
1     NaN  NaN  
2   11.70  

### Ahora se leen los csv de los años 2017 y 2018 para unificar con los datos anteriores y se guarda en un csv conjunto

El bucle es [17:19]

In [165]:
df_nuevos = pd.DataFrame()
for i in range (17,19)  :  
    fichero = '../dataCalidad/datos' + str(i) + '.csv'
    df_nuevo = pd.read_csv(fichero, delimiter = ';')
    df_nuevos = pd.concat([df_nuevos, df_nuevo])

In [166]:
print (df_nuevos.head())
print (df_nuevos.index.size)

   PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD PUNTO_MUESTREO   ANO  MES  D01  \
0         28         79         4         1  28079004_1_38  2017    1  6.0   
1         28         79         4         1  28079004_1_38  2017    2  8.0   
2         28         79         4         1  28079004_1_38  2017    3  5.0   
3         28         79         4         1  28079004_1_38  2017    4  4.0   
4         28         79         4         1  28079004_1_38  2017    5  4.0   

  V01  D02 ...   D27  V27  D28                  V28  D29  V29  D30      V30  \
0   V  8.0 ...   5.0    V  4.0                    V  4.0    V  5.0        V   
1   V  4.0 ...   4.0    V  4.0  V00000N00000N00000N  NaN  NaN  NaN      NaN   
2   V  7.0 ...   4.0    V  5.0                    V  7.0    V  7.0        V   
3   V  4.0 ...   3.0    V  4.0                    V  4.0    V  4.0  V00000N   
4   V  5.0 ...   5.0    V  5.0                    V  4.0    V  4.0        V   

   D31  V31  
0  7.0    V  
1  NaN  NaN  
2  5.0    V  


In [174]:
df_final.rename(columns={'AÑO': 'ANO'}, inplace=True)
df_final.rename(columns={'D1': 'D01', 'D2': 'D02', 'D3': 'D03', 'D4': 'D04', 'D5': 'D05', 'D6': 'D06', 'D7': 'D07' , 'D8': 'D08', 'D9': 'D09'}, inplace=True)
df_final.rename(columns={'V1': 'V01', 'V2': 'V02', 'V3': 'V03', 'V4': 'V04', 'V5': 'V05', 'V6': 'V06', 'V7': 'V07' , 'V8': 'V08', 'V9': 'V09'}, inplace=True)
print (df_final.head())
print (df_final.index.size)

  PROVINCIA MUNICIPIO  ESTACION  MAGNITUD PUNTO_MUESTREO  ANO  MES    D01 V01  \
0        29        79  28079003         1             NA   10    1  9.550   V   
1        29        79  28079003         1             NA   10    2  13.90   V   
2        29        79  28079003         1             NA   10    3  11.27   V   
3        29        79  28079003         1             NA   10    4  10.17   V   
4        29        79  28079003         1             NA   10    5  8.500   V   

     D02 ...     D27 V27    D28 V28                 D29  V29    D30  V30  \
0  11.23 ...   12.63   V  19.54   V               17.34    V  12.86    V   
1  19.53 ...   13.11   V  10.70   V  0.000N0.000N0.000N  NaN    NaN  NaN   
2  12.38 ...   11.49   V  13.05   V               12.48    V  11.50    V   
3  9.680 ...   9.680   V  9.200   V               8.930    V  9.140    V   
4  8.530 ...   7.820   V  7.090   V               8.100    V  8.470    V   

      D31  V31  
0   12.15    V  
1     NaN  NaN  
2   1

In [175]:
df_nuevos.index =range(df_final.index.size, (df_final.index.size + df_nuevos.index.size))
print (df_nuevos.head())
print (df_nuevos.index.size)

       PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD PUNTO_MUESTREO   ANO  MES  \
10662         28         79         4         1  28079004_1_38  2017    1   
10663         28         79         4         1  28079004_1_38  2017    2   
10664         28         79         4         1  28079004_1_38  2017    3   
10665         28         79         4         1  28079004_1_38  2017    4   
10666         28         79         4         1  28079004_1_38  2017    5   

       D01 V01  D02 ...   D27  V27  D28                  V28  D29  V29  D30  \
10662  6.0   V  8.0 ...   5.0    V  4.0                    V  4.0    V  5.0   
10663  8.0   V  4.0 ...   4.0    V  4.0  V00000N00000N00000N  NaN  NaN  NaN   
10664  5.0   V  7.0 ...   4.0    V  5.0                    V  7.0    V  7.0   
10665  4.0   V  4.0 ...   3.0    V  4.0                    V  4.0    V  4.0   
10666  4.0   V  5.0 ...   5.0    V  5.0                    V  4.0    V  4.0   

           V30  D31  V31  
10662        V  7.0    V  
10663   

In [176]:
df_unidos = pd.concat([df_final, df_nuevos])
print (df_unidos.head())
print (df_unidos.index.size)

  PROVINCIA MUNICIPIO  ESTACION  MAGNITUD PUNTO_MUESTREO  ANO  MES    D01 V01  \
0        29        79  28079003         1             NA   10    1  9.550   V   
1        29        79  28079003         1             NA   10    2  13.90   V   
2        29        79  28079003         1             NA   10    3  11.27   V   
3        29        79  28079003         1             NA   10    4  10.17   V   
4        29        79  28079003         1             NA   10    5  8.500   V   

     D02 ...     D27 V27    D28 V28                 D29  V29    D30  V30  \
0  11.23 ...   12.63   V  19.54   V               17.34    V  12.86    V   
1  19.53 ...   13.11   V  10.70   V  0.000N0.000N0.000N  NaN    NaN  NaN   
2  12.38 ...   11.49   V  13.05   V               12.48    V  11.50    V   
3  9.680 ...   9.680   V  9.200   V               8.930    V  9.140    V   
4  8.530 ...   7.820   V  7.090   V               8.100    V  8.470    V   

      D31  V31  
0   12.15    V  
1     NaN  NaN  
2   1

In [177]:
df_unidos.to_csv('../dataCalidad/datosCalidadUnidosFinal.csv')